In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import pyarrow.feather as feather
import os
from pathlib import Path

In [2]:
def apply_clustering_with_scaler(
    df, cols, algos, name, scaler=None, log1p=True, low=True
):
    """
    Applies clustering algorithms to each row of the DataFrame with optional scaling and transformation.

    Parameters:
    - df: DataFrame containing the data for clustering.
    - cols: List of column names to use for clustering.
    - algos: Dictionary of clustering algorithms to apply.
    - scaler: Scaler object (e.g., StandardScaler) to apply to the data.
    - log1p: Logical flag for log1p transformation before applying scaler.
    - low: Logical flag to indicate whether to identify the cluster with the lowest mean (True) or highest mean (False).

    Returns:
    - results_df: DataFrame with clustering results and metrics.
    """
    results = []

    for index, row in df.iterrows():
        # Prepare the data
        data = np.column_stack([row[col] for col in cols])
        true_labels = np.array([1] * row["p0"] + [0] * (row["p"] - row["p0"]))

        for algo_name, algorithm in algos.items():
            try:
                transformed_data = transform_data(data, scaler, log1p)

                # Fit the clustering algorithm
                if algo_name == "GaussianMixture":
                    algorithm.fit(transformed_data)
                    predicted_labels = algorithm.predict(transformed_data)
                else:
                    predicted_labels = algorithm.fit_predict(transformed_data)

                # Calculate cluster means and store them in a dictionary
                cluster_means = {
                    i: np.mean(data[:, 0][predicted_labels == i])
                    for i in np.unique(predicted_labels)
                }

                # Identify the cluster with low/high mean
                if low:
                    cluster_pos = min(cluster_means, key=cluster_means.get)
                else:
                    cluster_pos = max(cluster_means, key=cluster_means.get)

                # Assign label 1 to the selected cluster and label 0 otherwise
                predicted_labels = np.where(predicted_labels == cluster_pos, 1, 0)
                pos_idx = np.where(predicted_labels == 1)[0]

                # Evaluate the performance
                metrics = evaluate_clustering(true_labels, predicted_labels)
                results.append(
                    {
                        "dataset_name": row["dataset_name"],
                        "random_state": row["random_state"],
                        "SNR": row["SNR"],
                        "n": row["n"],
                        "p": row["p"],
                        "p0": row["p0"],
                        "pos_idx": pos_idx,
                        "Algorithm": name,
                        "Runtime": row["time_time"],
                        **metrics,
                    }
                )

            except Exception as e:
                # Catch exceptions for algorithms that fail
                results.append(
                    {
                        "dataset_name": row["dataset_name"],
                        "random_state": row["random_state"],
                        "SNR": row["SNR"],
                        "n": row["n"],
                        "p": row["p"],
                        "p0": row["p0"],
                        "pos_idx": pos_idx,
                        "Algorithm": name,
                        "Runtime": row["time_time"],
                        "Error": str(e),
                    }
                )

    # Create a DataFrame with the results
    results_df = pd.DataFrame(results)
    return results_df


def evaluate_clustering(true_labels, predicted_labels):
    tn, fp, fn, tp = confusion_matrix(true_labels, predicted_labels).ravel()
    tpr = tp / (tp + fn) * 100 if (tp + fn) > 0 else 0
    fpr = fp / (fp + tn) * 100 if (fp + tn) > 0 else 0
    fnr = fn / (fn + tp) * 100 if (fn + tp) > 0 else 0
    f1 = f1_score(true_labels, predicted_labels)
    return {"TPR": tpr, "FPR": fpr, "FNR": fnr, "F1": f1}


def evaluate_clustering_v2(pos_idx, p0, p):
    tp = sum(pos_idx < p0)
    fp = sum(pos_idx >= p0)
    fn = p0 - tp
    f1 = 2 * tp / (2 * tp + fp + fn)
    tpr = tp / p0 * 100
    fpr = fp / (p - p0) * 100
    fnr = fn / p0 * 100
    return {"TPR": tpr, "FPR": fpr, "FNR": fnr, "F1": f1}


def MPM(df, algo, col, one_row=True):
    """
    Variable selection via the Median Probability Model (MPM) criterion.

    Parameters:
    - df: DataFrame containing the data for clustering.
    - algos: Name of the algorithm.
    - col: Column name to use for clustering.

    Returns:
    - results_df: DataFrame with clustering results and metrics.
    """
    results = []

    for index, row in df.iterrows():
        # Prepare the data
        vip = row[col] if one_row else row[col][0, :]
        pos_idx = np.where(vip >= 0.5)[0]

        # Evaluate the performance
        metrics = evaluate_clustering_v2(pos_idx=pos_idx, p0=row["p0"], p=row["p"])
        results.append(
            {
                "dataset_name": row["dataset_name"],
                "random_state": row["random_state"],
                "SNR": row["SNR"],
                "n": row["n"],
                "p": row["p"],
                "p0": row["p0"],
                "pos_idx": pos_idx,
                "Algorithm": algo,
                "Runtime": row["time_time"],
                **metrics,
            }
        )

    # Create a DataFrame with the results
    results_df = pd.DataFrame(results)
    return results_df


def transform_data(data, scaler=None, log1p=True):
    """
    Applies a log1p transformation followed by optional scaling.

    Parameters:
    - data: 2D array-like data to transform.
    - scaler: Scaler object (e.g., StandardScaler) to apply.
    - log1p: Logical flag for log1p transformation before applying scaler

    Returns:
    - Transformed data.
    """

    # Log1p transformation
    transformed_data = np.log1p(data) if log1p else data

    # Apply scaler if provided
    if scaler is not None:
        transformed_data = scaler.fit_transform(transformed_data)

    return transformed_data


def add_summary_stats(df, columns, K=[20]):
    # Iterate over the dictionary to create new columns with the computed summaries
    new_cols = {}
    for k in K:
        # Define a dictionary with the summary statistic suffix and corresponding lambda function
        funcs = {
            "mu": lambda x: np.mean(x[:k, :], axis=0),
            "q25": lambda x: np.quantile(x[:k, :], q=0.25, axis=0),
            "q75": lambda x: np.quantile(x[:k, :], q=0.75, axis=0),
        }
        for col in columns:
            for suffix, func in funcs.items():
                new_col_name = f"{col}_{suffix}_K{k}"
                new_cols[new_col_name] = df[col].apply(func)

    new_df = pd.DataFrame(new_cols)
    df = pd.concat([df, new_df], axis=1)

    return df

In [3]:
algos = {
    "Agglomerative (average linkage)": AgglomerativeClustering(
        n_clusters=2, linkage="average"
    ),
}
notebook_dir = Path().resolve()
in_root_path = notebook_dir.parent / "results/no_idx/"
out_root_path = notebook_dir.parent / "results/with_idx/"

BART VC-measure toy example

In [13]:
df_BART = pd.read_feather(
    os.path.join(
        in_root_path,
        f"BART_VIP_Rank/feynman_BART_VIP_Rank_n1000_preidx.feather",
    )
)
df_BART = df_BART[
    (df_BART["SNR"] == 1)
    & (df_BART["dataset_name"] == "feynman_II_11_17")
    & (df_BART["random_state"] == 860)
]

# Convert list of 1D arrays into 2D arrays
df_BART["vip"] = df_BART["vip"].apply(lambda x: np.vstack(x))
df_BART["vip_rank"] = df_BART["vip_rank"].apply(lambda x: np.vstack(x))
df_BART["vc"] = df_BART["vc"].apply(lambda x: np.vstack(x))
df_BART["vc_rank"] = df_BART["vc_rank"].apply(lambda x: np.vstack(x))

# Calculate summary statistics
cols = ["vip_rank", "vip", "vc_rank", "vc"]
K = [10]
df_BART = add_summary_stats(df_BART, columns=cols, K=K)

# Remove raw columns
df_BART.drop(["vc", "vc_rank", "vip", "vip_rank", "s", "s_rank"], axis=1, inplace=True)

# BART VC-measure
BART_VC_measure = apply_clustering_with_scaler(
    df=df_BART,
    cols=["vc_mu_K10", "vc_q25_K10", "vc_rank_mu_K10", "vc_rank_q75_K10"],
    algos=algos,
    name="BART VC-measure",
    scaler=StandardScaler(),
    log1p=True,
    low=False,
)

BART_VC_measure_result = pd.merge(df_BART, BART_VC_measure, how="inner")

DART VC-measure and VIP-measure toy example

In [14]:
df_DART = pd.read_feather(
    os.path.join(
        in_root_path,
        f"DART_VC-measure/feynman_DART_VC-measure_n1000_preidx.feather",
    )
)
df_DART = df_DART[
    (df_DART["SNR"] == 1)
    & (df_DART["dataset_name"] == "feynman_II_11_17")
    & (df_DART["random_state"] == 860)
]

# Convert list of 1D arrays into 2D arrays
df_DART["vip"] = df_DART["vip"].apply(lambda x: np.vstack(x))
df_DART["vip_rank"] = df_DART["vip_rank"].apply(lambda x: np.vstack(x))
df_DART["vc"] = df_DART["vc"].apply(lambda x: np.vstack(x))
df_DART["vc_rank"] = df_DART["vc_rank"].apply(lambda x: np.vstack(x))

# Calculate summary statistics
cols = ["vip_rank", "vip", "vc_rank", "vc"]
K = [10]
df_DART = add_summary_stats(df_DART, columns=cols, K=K)

# Remove raw columns
df_DART.drop(["vc", "vc_rank", "vip", "vip_rank", "s", "s_rank"], axis=1, inplace=True)

# DART VC-measure
DART_VC_measure = apply_clustering_with_scaler(
    df=df_DART,
    cols=["vc_mu_K10", "vc_q25_K10", "vc_rank_mu_K10", "vc_rank_q75_K10"],
    algos=algos,
    name="DART VC-measure",
    scaler=StandardScaler(),
    log1p=True,
    low=False,
)

# DART VIP-measure
DART_VIP_measure = apply_clustering_with_scaler(
    df=df_DART,
    cols=["vip_mu_K10", "vip_q25_K10", "vip_rank_mu_K10", "vip_rank_q75_K10"],
    algos=algos,
    name="DART VIP-measure",
    scaler=StandardScaler(),
    log1p=True,
    low=False,
)

DART_VC_measure_result = pd.merge(df_DART, DART_VC_measure, how="inner")
DART_VIP_measure_result = pd.merge(df_DART, DART_VIP_measure, how="inner")

In [16]:
results = pd.concat(
    [BART_VC_measure_result, DART_VC_measure_result, DART_VIP_measure_result],
    ignore_index=True,
)
feather.write_feather(
    results,
    os.path.join(
        out_root_path,
        "results_toy_example.feather",
    ),
)